<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<h1> Exercise on behavior state effect on different brain regions </h1> 
    
<p> In the workshop today, we found two ways to define behavior state (an inutitive and an algorhmic approach). We also saw that the activity of some neurons is dependent upon the behavior state. This leads us to the question: which brain regions have the most cells which are sensitive to the behavior states that we found?
    
</div>

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<p> A simple question we can ask: are cells preferentially behavior state dependent the deeper you go? Do more superficial layers care less about the behavior state of the animal?
<div>

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
from scipy.stats import norm

pd.set_option('display.max_columns', None)

%matplotlib inline

import platform
platstring = platform.platform()

if 'Darwin' in platstring:
    # macOS 
    data_root = "/Volumes/Brain2023/"
elif 'Windows'  in platstring:
    # Windows (replace with the drive letter of USB drive)
    data_root = "E:/"
elif ('amzn' in platstring):
    # then on CodeOcean
    data_root = "/data/"
else:
    # then your own linux platform
    # EDIT location where you mounted hard drive
    data_root = "/media/$USERNAME/Brain2023/"

In [ ]:
from allensdk.brain_observatory.behavior.behavior_project_cache import VisualBehaviorNeuropixelsProjectCache

cache_dir  = data_root

cache = VisualBehaviorNeuropixelsProjectCache.from_local_cache(cache_dir=cache_dir, use_static_cache=True)

In [ ]:
area_of_interest = ['VISam', 'TH', 'DG', 'MGv'] # looking for regions with enhanced behavior state sensitivity
unit_nos = np.array([idx for idx, ccf in enumerate(unit_table.structure_acronym.values) if ccf in area_of_interest])
unit_ids = unit_table.index[unit_nos] 

# extract the sample unit ids and ready our three lists for recording values
example_unit_ids = unit_table.index
engaged_firing_rate_difference = []
unit_depth = []
in_region = []

# loop through all units
for n, unit in enumerate(example_unit_ids):
    unit_data = session.spike_times[unit]
    spike_times = [unit_data[np.logical_and(s1 <= unit_data, unit_data <= s2)] - s1 for s1, s2 in zip(start_times, stop_times)]
    
    # Compute the firing rate for each trial
    firing_rate = [len(spikes)/(s2 - s1) for spikes, s1, s2 in zip(spike_times, start_times, stop_times)]
    
    # compute the firing rate in both the engaged and disengaged trials
    engaged_firing_rate = []
    disengaged_firing_rate = []
    for i, trial in enumerate(switch_trials[:-1]):
        if not states.values[trial+1]: 
            engaged_firing_rate.append(firing_rate[i])
        else:
            disengaged_firing_rate.append(firing_rate[i])
            
    # identify if unit is in the your CCF.
    if unit in unit_ids:
        in_region.append(np.where(unit == np.array(unit_ids))[0][0])
    else:
        in_region.append(0)
    # Calculate the difference in firing rate for that unit between behavior states.
    engaged_firing_rate_difference.append((np.mean(engaged_firing_rate) - np.mean(disengaged_firing_rate)))
    unit_depth.append(unit_table.loc[unit].probe_vertical_position)

    
# make plot and prepare bins
fig,ax = plt.subplots(1,1, figsize = (4,3))    
NumBins = 12
bins = np.linspace(0, np.max(unit_depth), NumBins)
variance_at_depth = np.zeros([NumBins-1, 1])
for bin_idx in range(NumBins-1):
    keep_in_bin = np.logical_and(np.array(unit_depth) > bins[bin_idx], np.array(unit_depth) < bins[bin_idx + 1])
    variance_at_depth[bin_idx] = np.std(np.array(engaged_firing_rate_difference)[keep_in_bin])

ax.scatter(unit_depth, engaged_firing_rate_difference, c=in_region, alpha=0.1, cmap='jet')
ax.plot(bins[0:-1], variance_at_depth, 'g', lw=3)
#ax.set_xlim([0,1])
ax.set_ylim([-6,6])
ax.set_xlabel('Unit depth')
ax.set_ylabel('$\Delta$ Firing rate Engaged-Disengaged')
plt.tight_layout()
plt.show()

In [ ]:
fig,ax = plt.subplots(1,1, figsize = (4,3)) 
ax.scatter(unit_depth, engaged_firing_rate_difference, c=in_region, alpha=0.1, cmap='jet')
ax.plot(bins[0:-1], variance_at_depth, 'g', lw=3)
#ax.set_xlim([0,1])
ax.set_ylim([-6,6])
ax.set_xlabel('Unit depth')
ax.set_ylabel('$\Delta$ Firing rate Engaged-Disengaged')
plt.tight_layout()
plt.show()

Which direction does the probe measure?

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<p> In this simple study, we find that the response difference in firing rate of neurons across depth is seems to grow with depth. How could you validate this result with other data that you've seen? 
    
<p> Is this consistent with the thamlo-cortical loops which are often associated with being behavior state dependent?
</div>